In [ ]:
from inspect import isfunction, ismethod, isgeneratorfunction, isgenerator, isroutine
from inspect import isabstract, isclass, ismodule, istraceback, isframe, iscode, isbuiltin
from inspect import ismethoddescriptor, isdatadescriptor, isgetsetdescriptor, ismemberdescriptor
from inspect import isawaitable, iscoroutinefunction, iscoroutine

from collections.abc import Iterable as iterable

from pickle import load, dump

def isfx(field): return ismethod(field) or isfunction(field)

class GhostSet:
    """ enhanced interface (ghost) to retrieve class fields """
    def _meta(data): return {k:v for k,v in data.__dict__.items() if not isfx(v)}
    def _at_last(_, sets): pass
    def _set(object, **sets):
        ''' use to fast initialize fields | needed to avoid initialization problems at copy by value '''
        for field in sets: setattr(object, field, sets[field])
        object._at_last(sets)
GSet = GhostSet

def meta(object):
    ''' retrieves clonable object metadata (__dict__) as a copy '''
    if isinstance(object, GSet): return object._meta()
    return {}

class ClonableObjectGhost:
    """ enhanced interface (ghost) for clonable objects """
    def _by_val(_, depth=-1, _layer=0): pass
GCo = ClonableObjectGhost

class ClonableObject(GSet, GCo):
    """ base clonable object """
    def __init__(this, **data): this._set(**data)
    def __call__(_, **options): _._set(**options)
    def _by_val(_, depth=-1, _layer=0):
        copy = type(_)()
        copy._set(**_._meta())
        if depth<0 or depth>_layer:
            for field in copy.__dict__:
                if isinstance(copy.__dict__[field], ClonableObjectGhost):
                    copy.__dict__[field] = copy.__dict__[field]._by_val(depth,_layer+1)
        return copy
COb = ClonableObject

def copy_by_val(object, depth=-1, _layer=0):
    if isinstance(object, GCo): return object._by_val(depth,_layer)
    return object
copy = by_val = vof = copy_by_val

class ComparableGhost:
    """ enhanced interface (ghost) for comparing instances """
    def _compare(a, b):
        if type(a) != type(b): return False
        if a.__dict__ == b.__dict__: return True
        return False
    def __eq__(a, b): return a._compare(b)
GEq = ComparableGhost

class IterableObjectGhost(GSet):
    """ enhanced interface (ghost) for iterables: exposes __dict__,
        therefore Iterable Objects are like lua dictionaries """
    def __contains__(this, key): return key in this.__dict__
    def __iter__(this): return iter(this.__dict__)
    def items(my): return my.__dict__.items()
    def __getitem__(by, field): return by.__dict__[field]
    def __setitem__(by, field, value): by.__dict__[field] = value
    def pop(by, field): return by.__dict__.pop(field)
GIo = IterableObjectGhost

class ReprGhost:
    """ enhanced interface (ghost) for the skeleton method _repr,
        see implementation of Struct for a working example;
        Record __repr__ override uses _lines_ for max lines display """
    _lines_ = 31
    _chars_ = 13
    _msgsz_ = 62
    _ellipsis_ = ' ... '
    def _repr(my, value):
        _type = ''.join(''.join(str(type(value)).split('class ')).split("'"))
        _value = '{}'.format(value)
        if len(_value)>my._chars_:
            show = int(my._chars_/2)
            _value = _value[:show]+my._ellipsis_+_value[-show:]
        return '{} {}'.format(_type, _value)
    def _resize(this, message, at=.7):
        if len(message)>this._msgsz_:
            start = int(at*this._msgsz_)
            end = this._msgsz_-start
            return message[:start]+this._ellipsis_+message[-end:]
        return message
GRe = ReprGhost

def set_repr_to(lines): GRe._lines_ = lines

class Struct(COb, GEq, GIo, GRe):
    """ structured autoprintable object, behaves like a lua dictionary """
    def __repr__(_):
        return '\n'.join(['{}:\t{}'.format(k, _._repr(v)) for k,v in _.items()])
struct = Struct

class RecordableGhost:
    """ enhanced interface (ghost) for type recording,
        see Record for a working example """
    @staticmethod
    def load(filename):
        with open(filename, 'rb') as file: return load(file)
    def save(data, filename):
        with open(filename, 'wb') as file: dump(data, file)
        
GRec = RecordableGhost

class Record(GSet, GCo, GRec, GEq, GRe):
    """ wrapper for any object or value, auto-inspects and provides load/save type structure """
    data = None
    _check = dict(
            isfunction=isfunction, ismethod=ismethod, isgeneratorfunction=isgeneratorfunction, isgenerator=isgenerator, isroutine=isroutine,
            isabstract=isabstract, isclass=isclass, ismodule=ismodule, istraceback=istraceback, isframe=isframe, iscode=iscode, isbuiltin=isbuiltin,
            ismethoddescriptor=ismethoddescriptor, isdatadescriptor=isdatadescriptor, isgetsetdescriptor=isgetsetdescriptor, ismemberdescriptor=ismemberdescriptor,
            isawaitable=isawaitable, iscoroutinefunction=iscoroutinefunction, iscoroutine=iscoroutine
                   )
    def __init__(this, token, **meta):
        this.data = token
        this.__dict__.update({k:v(token) for k,v in this._check.items()})
        super()._set(**meta)
    @property
    def type(_): return type(_.data)
    def inherits(_, *types): return issubclass(_.type, types)
    @property
    def isbaseiterable(_): return _.inherits(tuple, list, dict, set) or _.isgenerator or _.isgeneratorfunction
    @property
    def isiterable(_): return isinstance(_.data, iterable) and _.type is not str
    def _clone_iterable(_):
        if _.inherits(dict): return _.data.copy()
        elif _.isgenerator or _.isgeneratorfunction: return (i for i in list(_.data))
        else: return type(_.data)(list(_.data)[:])
    def _meta(data): return {k:v for k,v in data.__dict__.items() if k != 'data' and not isfx(v)}
    def _by_val(_, depth=-1, layer=0):
        data = _.data
        if _.isiterable: data = _._clone_iterable()
        elif _.inherits(ClonableObjectGhost): data = by_val(data, depth, layer)
        return type(_)(data, **meta(_))
    def __enter__(self): self._instance = self; return self
    def __exit__(self, type, value, traceback): self._instance = None
    def __repr__(self):
        if not hasattr(self, '_preprint'): return Record(self.data, _preprint='', _lines=Record(Record._lines_)).__repr__()
        if self.isbaseiterable:
            pre, repr = self._preprint, ''
            for n,i in enumerate(self.data):
                if self._lines.data == 0: break
                else: self._lines.data -= 1
                index, item = str(n), i
                if self.inherits(dict): index += ' ({})'.format(str(i)); item = self.data[i]
                repr += pre+'{}: '.format(index)
                next = Record(item, _preprint=pre+'\t', _lines=self._lines)
                if next.isiterable: repr += '\n'
                repr += next.__repr__()
                repr += '\n'
            return repr
        elif self.inherits(GCo): return Record(self.data._meta(), _preprint=self._preprint, _lines=self._lines).__repr__()
        else: return self._repr(self.data)
REc = Record

class Bisect(list, COb):
    """ bisect implementation using clonable objects """
    def __init__(set, *items, key=None, reverse=False):
        if not key: key = lambda  x:x
        super().__init__(sorted(items, reverse=reverse, key=key))
    def _bisect(set, item, key, reverse, bottom, top):
        def _(check):
            if key: return key(check)
            return check
        at = int((top-bottom)/2)+bottom
        if len(set)==0: return (0,-1)
        if item==_(set[at]): return (at,0)
        bigger = item<_(set[at])
        if bigger != reverse:
            if at-bottom>0: return set._bisect(item, key, reverse, bottom, at)
            return (at,-1)
        elif top-at>1: return set._bisect(item, key, reverse, at, top)
        return (at,1)
    def search(_, item, key=None, reverse=False):
        if not key: key = lambda x:x
        return _._bisect(item, key, reverse, 0, len(_))
    def _by_val(_, depth=-1, _layer=0):
        copy = super()._by_val(depth, _layer)
        copy += _[:]
        return copy
BSx = Bisect

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
main_folder = "/home/kivi/gdrive/epigame-folder/"
path_res = main_folder + "cvs_pairs/"

woi = "preseizure1"
subjects = [id for id in range(1,22)]

In [ ]:
Subject, Pair, Labels, CM, CVS = [],[],[],[],[]

In [ ]:
for bands in [None,(0,4),(4,8),(8,13),(13,30)]:#[None,(0,4),(4,8),(8,13),(13,30),(30,70),(70,150)]:

    connectivity_measures = ["PAC", "PEC"] if bands is None else ["SCR", "SCI", "PLV", "PLI", "CC"]

    for measure in connectivity_measures:
        
        for subject_id in subjects:

            ext = "" if bands is None else f"-{bands}".replace(" ","")

            if bands is not None: filename = path_res + f"{subject_id}-{woi}-{measure}-{bands}.res".replace(" ","")
            elif bands is None: filename = path_res + f"{subject_id}-{woi}-{measure}.res"
            print(filename)

            res = REc.load(filename).data.data.pairs
            
            for pair_tuple in res:
                
                Subject.append(subject_id)   # e.g., 1
                Pair.append(pair_tuple[0])   # e.g., (0,1)
                Labels.append(pair_tuple[1]) # e.g., 'Q9-Q10<->U2-U3'
                CM.append(measure+ext)       # e.g., 'CC-(0,4)'
                CVS.append(pair_tuple[2])    # 1-D array

df = pd.DataFrame({"Subject":Subject, "Pair":Pair, "Labels":Labels, "CM":CM, "CVS":CVS})

df.to_csv(main_folder + "cvs_pairs_preseizure1.csv")

In [ ]:
df = pd.read_csv(main_folder + "cvs_pairs_preseizure1.csv")

In [ ]:
print(df[0:100])

In [ ]:
from random import randint, sample

def av(of): return sum(of)/len(of)
def mm(of): return min(of)*max(of)/av(of)

class Player:
    def __init__(my, AI, deck, name, n_in_hand=5):
        my.deck, my.n_in_hand, my.logic, my.cards, my.score, my.name = deck, n_in_hand, AI, [], [], name
    def shuffle(hand):
        lo, hi = min(hand.deck), max(hand.deck)
        hand.cards = sorted(sample(hand.deck, hand.n_in_hand)) # draw a number of random cards from the deck
    def check(scores): return 1+scores.real
    def play(god, *among):
        best, other = max(god.cards), []
        for player in among: other+=player.cards
        if best>=max(other): return best
        else:                return min(god.cards)
    def card(draw): return draw.logic(draw)

def rn_choice(logic):
    return logic.cards.pop(randint(0,len(logic.cards)-1))

def av_choice(logic):
    lo, hi = min(logic.deck), max(logic.deck)
    set, card, thresh = logic.cards, 0, (lo-1)+(1+hi-lo)/2
    actual_hand = av(logic.cards)
    if actual_hand>=thresh: card = set.pop(set.index(max(set)))
    else:                   card = set.pop(set.index(min(set)))
    return card

def mm_choice(logic):
    card, set = 0, logic.cards
    thresh, hi,lo = mm(set), max(set), min(set)
    if abs(hi-thresh)>abs(lo-thresh): card = set.pop(set.index(max(set)))
    else:                             card = set.pop(set.index(min(set)))
    return card

def mx_choice(logic):
    set = logic.cards
    return set.pop(set.index(max(set)))

def play(*game):
    for player in game: player.real,player.best=0,0; player.shuffle()
    def resolve(hand, by=[], best=-1):
        for player,card in hand:
            if best==card: by.append(player)
            if best <card: best=card; by=[player]
        return by
    winners = resolve([(player, player.card()) for player in game])
    for player in winners: player.real += 1
    for player in game: player.score.append(player.check())

In [ ]:
table_1 = df.groupby("Subject").get_group(1) # table is a subject

players = list(set(table_1.Pair))
print(players)

cvs = table_1.groupby("Pair").get_group(players[0]).groupby("CM").get_group("PEC").reset_index().CVS[0]
print(cvs)

cards = [float("0"+z) for z in [s for sub in [c.split("0") for c in cvs[1:-1].replace(" ","").split("\n")] for s in sub] if z!=""]
print(cards)

In [ ]:
table_1 = df.groupby("Subject").get_group(1) # table is a subject
conn_measures = list(set(table_1.CM))
players = list(set(table_1.Pair))

game = []
player_deck = {pair:[] for pair in players}

for node_pair in players:

    for connectivity in conn_measures:

        cvs = table_1.groupby("Pair").get_group(node_pair).groupby("CM").get_group(connectivity).reset_index().CVS[0]

        ones = cvs.count("1.")
        cvs = cvs.replace("1.","")

        a = [x for y in [c.split("0") for c in cvs[1:-1].replace(" ","").split("\n")] for x in y]
        cards=[1.0 for i in range(ones)]
        for s in a:
            if s=="": pass
            elif s[-2::]=="1.": cards.append(float("0"+s[:-2])); cards.append(float(s[-2::]))
            else: cards.append(float("0"+s))

        player_deck[node_pair]+=cards

In [ ]:
from scipy.interpolate import interp1d

remap = interp1d([0,1],[1,100])

deck_remapped = {node_pair:[float(remap(val)) for val in player_deck[node_pair]] for node_pair in player_deck}

strategy = mm_choice

test = {p:[] for p in players}

rounds = 10

while rounds:

    game = [Player(strategy, deck_remapped[p], p) for p in players]

    for j in range(10):
        n_cards = 24
        for player in game: player.n_in_hand = n_cards
        play(*game)

    scores = sorted([(player.name, player.score) for player in game], key=lambda x:x[1], reverse=True)
    print(scores)

    top_score, fall = scores[0][1], 1
    for name, score in scores:
        if score==top_score: test[name].append(1); fall+=1
        elif score!=top_score: break
    for name,score in scores[fall:]: test[name].append(0)
    print(test)
    
    rounds -= 1

In [ ]:
from pickle import load, dump

x=dump(test, open(main_folder + "test_preseizure1_sub1.p", "wb"))